In [ ]:
import pandas as pd
import tensorflow as tf

data = pd.read_csv('../input/titanic/train.csv')
data.pop(item='Name')
data.pop(item='Ticket')
data.pop(item='PassengerId')
y_train = data.pop(item='Survived')
s = pd.get_dummies(data['Sex'])
pclass = pd.get_dummies(data['Pclass'], prefix="class")
data.pop(item='Sex')
data.pop(item='Pclass')
data.pop(item='Embarked')
data = data.join(s)
data = data.join(pclass)
data.pop(item='Cabin')
data['Age'].fillna(0, inplace=True)
X = data.values
X[:,0] = tf.keras.utils.normalize(X[:,0])
X[:,3] = tf.keras.utils.normalize(X[:,3])
X.shape, data

In [ ]:
pd.DataFrame(X)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(50, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy']
              )
history = model.fit(X[:712], y_train[:712], epochs=100, batch_size=X.size)


In [ ]:
import matplotlib.pyplot as plt
pd.DataFrame(history.history).plot()

In [ ]:
model.evaluate(X[712:], y_train[712:])

In [ ]:
data2 = pd.read_csv('../input/titanic/test.csv')
data2.pop(item='Name')
data2.pop(item='Ticket')
data2.pop(item='PassengerId')
s = pd.get_dummies(data2['Sex'])
pclass = pd.get_dummies(data2['Pclass'], prefix="class")
data2.pop(item='Sex')
data2.pop(item='Pclass')
data2.pop(item='Embarked')
data2 = data2.join(s)
data2 = data2.join(pclass)
data2.pop(item='Cabin')
data2['Age'].fillna(0, inplace=True)
data2['Fare'].fillna(0, inplace=True)
X_test = data2.values
X_test[:,0] = tf.keras.utils.normalize(X_test[:,0])
X_test[:,3] = tf.keras.utils.normalize(X_test[:,3])
X_test.shape, data2

In [ ]:
X_test

In [ ]:
predictions = model.predict(X_test)
predictions = tf.round(predictions)
predictions = tf.squeeze(predictions)
predictions = tf.cast(predictions, tf.int32)
predictions.dtype


In [ ]:
d = {'PassengerId': [i for i in range(892,892+418)], 'Survived' : predictions }
df = pd.DataFrame(d)
df.to_csv('submission.csv', index=False)